<a href="https://colab.research.google.com/github/proninivandm/Agro_code/blob/main/Agrocode_%D0%9A%D1%83%D0%BB%D1%8C%D1%82%D1%83%D1%80%D1%8B.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install catboost

     |████████████████████████████████| 66.3MB 70kB/s 


In [ ]:
import pandas as pd
from sklearn.impute import KNNImputer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score, classification_report
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import catboost
RANDOM_SEED = 10

In [ ]:
def preproc(df):
  diffs_df = df[days_cols].diff(axis=1).drop('day_1', axis=1)
  residuals_df = days_mean_df - df[days_cols]
  residuals_med_df = days_median_df - df[days_cols]
  diffs_df.rename(columns=lambda x: x.replace('day', 'diff'), inplace=True)
  residuals_df.rename(columns=lambda x: x.replace('day', 'res'), inplace=True)
  residuals_med_df.rename(columns=lambda x: x.replace('day', 'res_med'), inplace=True)
  df = pd.concat([df, diffs_df, residuals_df, residuals_med_df], axis=1)
  df['vmax'] = df[days_cols].max(axis=1)
  df['vmin'] = df[days_cols].min(axis=1)
  df['dmax'] = diffs_df.max(axis=1)
  df['dmin'] = diffs_df.min(axis=1)
  df['rmax'] = residuals_df.max(axis=1)
  df['rmin'] = residuals_df.min(axis=1)
  df['rmmax'] = residuals_med_df.max(axis=1)
  df['rmmin'] = residuals_med_df.min(axis=1)
  df['vdiff'] = df.vmax-df.vmin
  df['ddiff'] = df.dmax-df.dmin
  df['rdiff'] = df.rmax-df.rmin
  df['rmdiff'] = df.rmmax-df.rmmin
  df['argvmax'] = df.apply(lambda x: x[days_cols].argmax(), axis=1)
  df['argvmin'] = df.apply(lambda x: x[days_cols].argmin(), axis=1)
  df['argdmax'] = diffs_df.apply(lambda x: x.argmax(), axis=1)
  df['argdmin'] = diffs_df.apply(lambda x: x.argmin(), axis=1)
  df['argrmax'] = residuals_df.apply(lambda x: x.argmax(), axis=1)
  df['argrmin'] = residuals_df.apply(lambda x: x.argmin(), axis=1)
  df['argrmmax'] = residuals_med_df.apply(lambda x: x.argmax(), axis=1)
  df['argrmmin'] = residuals_med_df.apply(lambda x: x.argmin(), axis=1)
  df['vmean'] = df.apply(lambda x: x[days_cols].mean(), axis=1)
  df['rmean'] = residuals_df.apply(lambda x: x.mean(), axis=1)
  df['rmmean'] = residuals_med_df.apply(lambda x: x.mean(), axis=1)
  df['vvar'] = df.apply(lambda x: x[days_cols].var(), axis=1)
  df['dvar'] = diffs_df.apply(lambda x: x.var(), axis=1)
  df['rvar'] = residuals_df.apply(lambda x: x.var(), axis=1)
  df['rmvar'] = residuals_med_df.apply(lambda x: x.var(), axis=1)
  return df

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
train = pd.read_csv('/content/drive/MyDrive/train.csv')
test = pd.read_csv('/content/drive/MyDrive/test.csv')
example = pd.read_csv('/content/drive/MyDrive/example_csv_agrocode-task1.csv')

print(train.shape, test.shape)

(2838, 370) (939, 369)


In [ ]:
train.rename(columns=lambda x: x.replace(' ', '_').lower(), inplace=True)
train = train.drop('day_366', axis=1)
test.rename(columns=lambda x: x.replace(' ', '_').lower(), inplace=True)
test = test.drop('day_366', axis=1)

days_cols = [x for x in train.columns if 'day' in x]


In [ ]:
days_mean_df = pd.concat([train.drop('culture', axis=1), test], axis=0)[days_cols].apply(np.mean, axis=0)
days_median_df = pd.concat([train.drop('culture', axis=1), test], axis=0)[days_cols].apply(np.median, axis=0)

In [ ]:
to_fit=pd.get_dummies(pd.concat([train.drop('culture', axis=1), test], axis=0), columns=['year', 'field_id'])

In [ ]:
%%time
imputer = KNNImputer(n_neighbors=9, weights="distance")
imputer.fit(to_fit)

CPU times: user 48.4 ms, sys: 26.4 ms, total: 74.8 ms
Wall time: 79.3 ms


In [ ]:
to_fit[:len(train)] = imputer.transform(to_fit[:len(train)])
train[days_cols] = to_fit[:len(train)][days_cols]

In [ ]:
to_fit[-len(test):] = imputer.transform(to_fit[-len(test):])
test[days_cols] = to_fit[-len(test):][days_cols]

In [ ]:
train_new = preproc(train)
test_new = preproc(test)

In [ ]:
to_delete = [28,18,25,19,26,27]
train_new = train_new.drop(train_new.query('culture in {}'.format(to_delete)).index)

In [ ]:
un = train_new.culture.unique()

In [ ]:
x = range(len(un))

In [ ]:
labels = {sorted(un)[x]:x for x in x}

In [ ]:
labels

{1: 0,
 2: 1,
 3: 2,
 4: 3,
 5: 4,
 6: 5,
 7: 6,
 8: 7,
 9: 8,
 10: 9,
 11: 10,
 12: 11,
 13: 12,
 14: 13,
 15: 14,
 16: 15,
 17: 16,
 20: 17,
 21: 18,
 22: 19,
 23: 20,
 24: 21}

In [ ]:
train_new.culture=train_new.culture.apply(labels.get)

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(train_new.drop('culture', axis=1), train_new.culture, random_state=RANDOM_SEED)

In [ ]:
train_pool = catboost.Pool(X_train, y_train, ['year', 'field_id'])
val_pool = catboost.Pool(X_val, y_val, ['year', 'field_id'])
test_pool = catboost.Pool(test_new, cat_features = ['year', 'field_id'])
full_pool = catboost.Pool(train_new.drop('culture', axis=1), train_new.culture, cat_features = ['year', 'field_id'])

In [ ]:
cat_model = catboost.CatBoostClassifier(loss_function='MultiClass', eval_metric='TotalF1:average=Weighted', classes_count=len(train_new.culture.unique()), task_type='GPU', random_state=RANDOM_SEED, iterations=2000)

In [ ]:
cat_model.fit(train_pool, eval_set=val_pool, use_best_model=True, metric_period=50)

In [ ]:
cat_model.fit(full_pool, metric_period=50)

Learning rate set to 0.045043
0:	learn: 0.5129877	total: 69.7ms	remaining: 2m 19s
50:	learn: 0.8097069	total: 3.6s	remaining: 2m 17s
100:	learn: 0.8675380	total: 6.93s	remaining: 2m 10s
150:	learn: 0.8974469	total: 10.1s	remaining: 2m 4s
200:	learn: 0.9199869	total: 13.2s	remaining: 1m 58s
250:	learn: 0.9366943	total: 16.3s	remaining: 1m 53s
300:	learn: 0.9506863	total: 19.4s	remaining: 1m 49s
350:	learn: 0.9577559	total: 22.4s	remaining: 1m 45s
400:	learn: 0.9680241	total: 25.5s	remaining: 1m 41s
450:	learn: 0.9716549	total: 28.5s	remaining: 1m 37s
500:	learn: 0.9786330	total: 31.5s	remaining: 1m 34s
550:	learn: 0.9838673	total: 34.6s	remaining: 1m 31s
600:	learn: 0.9873230	total: 37.6s	remaining: 1m 27s
650:	learn: 0.9892035	total: 40.6s	remaining: 1m 24s
700:	learn: 0.9906388	total: 43.6s	remaining: 1m 20s
750:	learn: 0.9913916	total: 46.7s	remaining: 1m 17s
800:	learn: 0.9932396	total: 49.7s	remaining: 1m 14s
850:	learn: 0.9951561	total: 52.7s	remaining: 1m 11s
900:	learn: 0.995156

In [ ]:
cv_model = catboost.cv(full_pool, params = cat_model.get_params(), fold_count=5, partition_random_seed=RANDOM_SEED, metric_period = 50, plot=True)

In [ ]:
cv_model

,iterations,test-TotalF1:average=Weighted-mean,test-TotalF1:average=Weighted-std,train-TotalF1:average=Weighted-mean,train-TotalF1:average=Weighted-std,test-MultiClass-mean,test-MultiClass-std,train-MultiClass-mean,train-MultiClass-std
0,0,0.447672,0.057025,0.510973,0.003724,2.994055,0.026986,2.936852,0.007742
1,50,0.716477,0.014413,0.765940,0.005688,1.131360,0.032245,1.024015,0.006218
2,100,0.757519,0.019840,0.832132,0.007606,0.844550,0.026860,0.705798,0.006149
3,150,0.785634,0.025300,0.867254,0.006985,0.738326,0.026934,0.572996,0.005352
4,200,0.802759,0.019291,0.889445,0.006436,0.676416,0.026008,0.489646,0.004656
5,250,0.813601,0.015779,0.904525,0.003882,0.633052,0.025754,0.428618,0.003955
6,300,0.823981,0.016130,0.919117,0.002660,0.601287,0.026296,0.382132,0.002879
7,350,0.831690,0.016551,0.931622,0.003623,0.576720,0.024922,0.344163,0.002852
8,400,0.837467,0.014399,0.941510,0.002687,0.556282,0.024525,0.312705,0.002896
9,450,0.842106,0.012348,0.951222,0.002140,0.539810,0.024350,0.286775,0.003276


In [ ]:
train_new.culture.value_counts()

In [ ]:
train.groupby('culture').mean()

,year,field_id,field_area,day_1,day_2,day_3,day_4,day_5,day_6,day_7,day_8,day_9,day_10,day_11,day_12,day_13,day_14,day_15,day_16,day_17,day_18,day_19,day_20,day_21,day_22,day_23,day_24,day_25,day_26,day_27,day_28,day_29,day_30,day_31,day_32,day_33,day_34,day_35,day_36,day_37,...,day_326,day_327,day_328,day_329,day_330,day_331,day_332,day_333,day_334,day_335,day_336,day_337,day_338,day_339,day_340,day_341,day_342,day_343,day_344,day_345,day_346,day_347,day_348,day_349,day_350,day_351,day_352,day_353,day_354,day_355,day_356,day_357,day_358,day_359,day_360,day_361,day_362,day_363,day_364,day_365
culture,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,2016.416667,413.888889,129.942500,0.076522,0.073978,0.071453,0.068919,0.066397,0.064133,0.061844,0.059617,0.056011,0.052428,0.048956,0.046983,0.045108,0.043286,0.041614,0.040064,0.038661,0.037397,0.036267,0.035289,0.034486,0.033756,0.033017,0.032253,0.031525,0.031022,0.031086,0.031172,0.031289,0.031411,0.031567,0.031744,0.031947,0.032192,0.032497,0.032844,0.033247,...,0.277895,0.268404,0.258168,0.247807,0.237204,0.226742,0.222583,0.221175,0.220028,0.215078,0.210083,0.205081,0.200072,0.195051,0.190020,0.184959,0.179782,0.174500,0.165512,0.156817,0.148785,0.141657,0.134487,0.127480,0.120757,0.116453,0.113939,0.111864,0.109802,0.107791,0.106230,0.109292,0.112483,0.115558,0.118639,0.121724,0.124807,0.126502,0.128206,0.127855
2,2016.836735,424.877551,140.297347,0.064800,0.061704,0.058516,0.055245,0.052120,0.049039,0.046002,0.043039,0.037667,0.032300,0.027692,0.023549,0.019788,0.016243,0.013198,0.010369,0.007496,0.004651,0.001853,-0.000939,-0.002482,-0.003924,-0.005271,-0.006510,-0.007635,-0.008661,-0.008049,-0.007112,-0.006088,-0.005035,-0.003922,-0.002796,-0.001667,-0.000424,0.000784,0.001992,0.003182,...,0.384020,0.367641,0.351284,0.334355,0.317757,0.301543,0.295553,0.291978,0.288304,0.283227,0.278122,0.272990,0.267820,0.262610,0.257386,0.252135,0.246878,0.241412,0.232581,0.225314,0.217983,0.210659,0.203369,0.195740,0.188165,0.180671,0.173990,0.167353,0.160741,0.154166,0.147613,0.150747,0.153866,0.156977,0.160076,0.163122,0.163874,0.164600,0.165271,0.164563
3,2016.961538,889.026627,127.100473,0.124676,0.124095,0.123459,0.122778,0.122261,0.121765,0.121167,0.120098,0.117372,0.115136,0.113400,0.111825,0.110201,0.108457,0.105277,0.101847,0.098494,0.095213,0.092258,0.089350,0.087740,0.086636,0.085452,0.084445,0.083170,0.082028,0.082127,0.082171,0.082735,0.083493,0.084530,0.086034,0.087336,0.088538,0.089609,0.090642,0.091679,...,0.241282,0.236273,0.231547,0.227901,0.225041,0.222311,0.220012,0.218276,0.216881,0.215704,0.214576,0.214475,0.214382,0.214294,0.214190,0.214071,0.213946,0.213476,0.210000,0.209240,0.208039,0.206245,0.204116,0.201849,0.198880,0.195976,0.193348,0.190769,0.188329,0.186019,0.183830,0.181794,0.179898,0.178251,0.174301,0.170247,0.166735,0.163458,0.160160,0.156646
4,2017.235972,799.308993,96.159400,0.319594,0.318710,0.317786,0.316924,0.316565,0.316273,0.315914,0.314758,0.311697,0.307930,0.305630,0.303421,0.301345,0.299256,0.296137,0.292228,0.288404,0.284722,0.281017,0.277368,0.276183,0.275542,0.274618,0.274424,0.273859,0.273666,0.274087,0.274659,0.276752,0.278863,0.281169,0.283771,0.285924,0.287746,0.289424,0.290865,0.292252,...,0.251860,0.247500,0.243124,0.239010,0.236564,0.233578,0.231130,0.228942,0.226792,0.224711,0.222714,0.223022,0.223893,0.224780,0.225672,0.227292,0.229076,0.230377,0.232334,0.233274,0.234582,0.233797,0.231955,0.229803,0.226347,0.223137,0.220112,0.217000,0.213877,0.210765,0.207565,0.204558,0.201972,0.199803,0.194520,0.189276,0.184405,0.180316,0.176098,0.171589
5,2017.151515,823.030303,126.761742,0.080490,0.077426,0.074350,0.071263,0.068223,0.065493,0.062789,0.060161,0.056194,0.051240,0.046458,0.042215,0.038009,0.033836,0.029702,0.025595,0.021512,0.017473,0.013537,0.009534,0.005652,0.001921,-0.001708,-0.005147,-0.007740,-0.009152,-0.009193,-0.008949,-0.008731,-0.008505,-0.008287,-0.008048,-0.007784,-0.007500,-0.007164,-

In [ ]:
0.876496

In [ ]:
test['Culture'] = cat_model.predict(test_pool)

In [ ]:
labels = {k:v for v,k in labels.items()}

In [ ]:
labels

{0: 1,
 1: 2,
 2: 3,
 3: 4,
 4: 5,
 5: 6,
 6: 7,
 7: 8,
 8: 9,
 9: 10,
 10: 11,
 11: 12,
 12: 13,
 13: 14,
 14: 15,
 15: 16,
 16: 17,
 17: 20,
 18: 21,
 19: 22,
 20: 23,
 21: 24}

In [ ]:
test.Culture = test.Culture.apply(labels.get)

In [ ]:
test.to_csv('preds9_cfull_pool.csv', index=False, columns=['Culture'])

In [ ]:
len(test)

In [ ]:
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import torch
from torch import distributions,optim
from sklearn.base import BaseEstimator
from torch.autograd import Variable
torch.random.manual_seed(RANDOM_SEED)

In [ ]:
class EncoderRNN(nn.Module):
    def __init__(self, n_features, latent_dim, hidden_size):
      super(EncoderRNN, self).__init__()

      self.n_features = n_features
      self.hidden_size = hidden_size
      self.latent_dim = latent_dim
      self.lstm_enc = nn.LSTM(n_features, hidden_size,
                      batch_first = True,dropout=0.1,
                      bidirectional=False)
    
def forward(self, x):
    x, h  = self.lstm_enc(x)
    return x

In [ ]:
class DecoderRNN(nn.Module):
  def __init__(self, seq_len, n_features, latent_dim , hidden_size):
    super(DecoderRNN, self).__init__()

    self.seq_len = seq_len
    self.n_features = n_features
    self.latent_dim = latent_dim
    self.hidden_size = hidden_size

    self.lstm_dec1 = nn.LSTM(latent_dim, latent_dim,
                     batch_first = True, dropout=0.1,
                     bidirectional= False)


    self.output_layer = nn.Linear(self.hidden_size, n_features,bias=True)

  def forward(self, x):
   x = x.repeat(1,self.seq_len, 1)
   x, _ = self.lstm_dec1(x)
   return self.output_layer(x)

In [ ]:
class AERNN(nn.Module):
  def __init__(self, seq_len, n_features, latent_dim , hidden_size):
    super(AERNN, self).__init__()

    self.seq_len = seq_len
    self.encoder = EncoderRNN(n_features, latent_dim, hidden_size).to(device)
    self.decoder = DecoderRNN(seq_len, n_features, latent_dim, hidden_size).to(device)

  def forward(self, x):
    x = self.encoder.forward(x)
    x = self.decoder.forward(x)
    return x

In [ ]:
class cult_dataset(Dataset):
  def __init__(self, x, transform=None):
        """
        Args:
            csv_file (string): Path to the csv file with annotations.
            root_dir (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        self.x = x
        self.transform = transform

  def __len__(self):
    return len(self.x)

  def __getitem__(self, idx):
      if torch.is_tensor(idx):
          idx = idx.tolist()

      sample=self.x.iloc[idx]
        
      if self.transform:
          sample = self.transform(sample)

      return torch.tensor(sample)

In [ ]:
days_dataset=cult_dataset(to_fit[days_cols])

In [ ]:
device=torch.device('cuda')

In [ ]:
model = AERNN(365, 1, 100 , 50)
model = model.to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
loss_fn = nn.MSELoss().to(device)

/usr/local/lib/python3.6/dist-packages/torch/nn/modules/rnn.py:61: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.1 and num_layers=1
  "num_layers={}".format(dropout, num_layers))


In [ ]:
dl = DataLoader(days_dataset, batch_size=256, shuffle=True)

In [ ]:
    for epoch in range(100):
        for x  in dl:
            x = x.cuda()
            optimizer.zero_grad()
            recon = model.forward(x)
            loss = loss_fn(recon, x)
            loss.backward()
            optimizer.step()
        print(loss.cpu().item(), '  epoch', epoch)

NotImplementedError: ignored